### While Loops — Advanced Patterns (practical, not too insane)

These patterns build on the basics: `while` conditions, guards to avoid infinite loops, `while`–`else`, input loops, backoff retries, queue/stack draining, two-pointer scans, and timeouts.

## 1) Emulating *do–while*: run body at least once

In [1]:
x = 0
while True:
    x += 1  # body runs at least once
    print("tick", x)
    if x >= 3:
        break  # exit condition checked after body

tick 1
tick 2
tick 3


## 2) `while`–`else`: run `else` only if loop didn't `break`

In [2]:
nums = [4, 8, 12, 16]
i = 0
while i < len(nums):
    if nums[i] % 2:  # odd -> break
        print("found odd", nums[i])
        break
    i += 1
else:  # reached naturally (no break)
    print("all numbers were even")

all numbers were even


## 3) Guard against infinite loops with a maximum-steps safety valve

In [3]:
state = 0
max_steps = 10_000
steps = 0
while state != 5:
    state += 1  # pretend: complex transition
    steps += 1
    if steps > max_steps:
        raise RuntimeError("loop safety valve tripped")
print("done with state=", state)

done with state= 5


## 4) Time-based timeout using `time.monotonic()` (robust vs system clock changes)

In [4]:
import time

deadline = time.monotonic() + 1.5  # seconds from now
attempts = 0
ok = False
while time.monotonic() < deadline:
    attempts += 1
    # fake condition; replace with real readiness check
    if attempts == 3:
        ok = True
        break
    time.sleep(0.2)
print({"ok": ok, "attempts": attempts})

{'ok': True, 'attempts': 3}


## 5) Exponential backoff with jitter (retry pattern)

In [5]:
import random, time

retries = 0
max_retries = 5
base = 0.1  # seconds
success = False

while retries < max_retries:
    # call flaky operation (here simulated)
    success = random.random() < 0.35
    if success:
        break
    delay = base * (2 ** retries) + random.uniform(0, base)
    time.sleep(delay)
    retries += 1

print({"success": success, "retries": retries})

{'success': True, 'retries': 0}


## 6) Input validation loop using the walrus operator (`:=`)

In [6]:
# In notebooks, replace input() by a list iterator for demo
samples = iter(["abc", "-3", "42"])  # pretend these came from input()

while True:
    try:
        s = next(samples)  # s := input("Enter a positive int: ")
    except StopIteration:
        print("no valid input provided")
        break
    if s.isdigit() and int(s) > 0:
        print("accepted:", int(s))
        break
    print("invalid:", s)

invalid: abc
invalid: -3
accepted: 42


## 7) Draining a stack / queue with `while` (mutation-friendly)

In [7]:
from collections import deque

# Stack (LIFO)
stack = [1, 2, 3]
while stack:
    x = stack.pop()  # last-in first-out
    print("stack popped", x)

# Queue (FIFO)
q = deque(["a", "b", "c"])
while q:
    x = q.popleft()
    print("queue got ", x)

stack popped 3
stack popped 2
stack popped 1
queue got  a
queue got  b
queue got  c


## 8) Two-pointer technique with `while` (merge two sorted lists)
Efficient O(n+m) merge used in mergesort and joining sorted streams.

In [8]:
a = [1, 3, 7, 10]
b = [2, 3, 6, 11]
i = j = 0
merged = []

while i < len(a) and j < len(b):
    if a[i] <= b[j]:
        merged.append(a[i]); i += 1
    else:
        merged.append(b[j]); j += 1

# append any remainder
while i < len(a):
    merged.append(a[i]); i += 1
while j < len(b):
    merged.append(b[j]); j += 1

print(merged)

[1, 2, 3, 3, 6, 7, 10, 11]


## 9) Reading lines until EOF with assignment expression
This mirrors typical stream processing without loading entire file into memory.

In [9]:
# Prepare a demo file
path = "_demo_while_read.txt"
with open(path, "w", encoding="utf-8") as f:
    f.write("alpha\n\n beta\n gamma \n")

clean = []
with open(path, encoding="utf-8") as f:
    line = f.readline()
    while line:  # non-empty string means not EOF
        s = line.strip()
        if s:
            clean.append(s)
        line = f.readline()
print(clean)

['alpha', 'beta', 'gamma']


## 10) Polling with incremental sleep (soft CPU usage)

In [10]:
import time

ready = False
sleep_s = 0.05
max_sleep = 0.4
attempts = 0
start = time.monotonic()

while not ready and (time.monotonic() - start) < 1.0:
    attempts += 1
    # fake readiness after a few tries
    if attempts == 4:
        ready = True
        break
    time.sleep(sleep_s)
    sleep_s = min(max_sleep, sleep_s * 2)  # gradually relax polling

print({"ready": ready, "attempts": attempts})

{'ready': True, 'attempts': 4}


## 11) Finite-state style loop (simple numeric lexer)
Recognize contiguous digits and emit them as integers; skip other chars.

In [11]:
s = "abc12d003x9"
i = 0
out = []
while i < len(s):
    if s[i].isdigit():
        j = i
        while j < len(s) and s[j].isdigit():
            j += 1
        out.append(int(s[i:j]))
        i = j
    else:
        i += 1
print(out)  # [12, 3, 9]

[12, 3, 9]


## 12) Robust removal while iterating (index-based from end)
Modify a list in-place by walking indices **backwards** to avoid skipping elements.

In [12]:
vals = [1, None, 2, None, 3, None]
i = len(vals) - 1
while i >= 0:
    if vals[i] is None:
        vals.pop(i)
    i -= 1
print(vals)  # [1, 2, 3]

[1, 2, 3]


## 13) Mini-exercises
1. Implement a `while` loop that keeps dividing `n` by 2 (integer division) until it reaches 0; count steps.
2. Write a retry loop with a fixed timeout and **max attempts**; ensure both guards are enforced.
3. Use a two-pointer `while` to check if a string is a palindrome, ignoring non-alphanumerics.
4. Drain a `deque` of tasks where each task may append new tasks (simulate breadth-first processing).